In [ ]:
pip install pillow exif

In [ ]:
# Extract the GPS data from images
from PIL import Image
from exif import Image as ExifImage
import os

def get_gps_coordinates(image_path):
    with open(image_path, 'rb') as image_file:
        my_image = ExifImage(image_file)
        if my_image.has_exif and hasattr(my_image, 'gps_latitude') and hasattr(my_image, 'gps_longitude'):
            return my_image.gps_latitude, my_image.gps_longitude
        else:
            return None, None

image_directory = r"/content/drive/MyDrive/Colab Notebooks/EfficientDet-Lite4/dataset/images"
gps_data = []

for filename in os.listdir(image_directory):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):  # Add other file types if necessary
        file_path = os.path.join(image_directory, filename)
        lat, lon = get_gps_coordinates(file_path)
        if lat and lon:
            gps_data.append((lat, lon))



In [ ]:
# save the GPS in Csv file
import csv

with open('gps_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Latitude', 'Longitude'])
    for data in gps_data:
        writer.writerow(data)


In [ ]:
pip install folium

def dms_to_decimal(degrees, minutes, seconds):
    return degrees + minutes / 60 + seconds / 3600


In [ ]:
# Changethe format of the GPS data from degrees to decimals
import csv
from ast import literal_eval

def dms_to_decimal(degrees, minutes, seconds):
    return degrees + minutes / 60 + seconds / 3600

converted_gps_data = []

with open('gps_data.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip header row
    for row in reader:
        # Parse the DMS strings to tuples
        lat_dms = literal_eval(row[0])
        lon_dms = literal_eval(row[1])
        # Convert DMS to decimal degrees
        decimal_lat = dms_to_decimal(*lat_dms)
        decimal_lon = dms_to_decimal(*lon_dms)
        converted_gps_data.append((decimal_lat, decimal_lon))


In [ ]:
# Create the heat map
import folium
from folium.plugins import HeatMap

# Calculate the average latitude and longitude for the map center
average_latitude = sum(lat for lat, _ in converted_gps_data) / len(converted_gps_data)
average_longitude = sum(lon for _, lon in converted_gps_data) / len(converted_gps_data)

# Create a base map
map = folium.Map(location=[average_latitude, average_longitude], zoom_start=12)

# Add HeatMap
HeatMap(converted_gps_data).add_to(map)

# Save or display the map
map.save('heatmap.html')